<a href="https://colab.research.google.com/github/tolani007/Fun-Data-Science-Content-from-Tiki/blob/main/Eigentiki's_first_PYSPARK_PROJECT_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I built a Content Recommendation Engine using the Alternating Least Squares (ALS) algorithm for fun, s/o N8N's dad joor.

MY GOAL is toBuild and evaluate a collaborative filtering model in < 60 minutes.

I will simulate 1,000 users interacting with 500 content pieces (articles/puzzles).

In [1]:
!pip install pyspark

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, col
import random


#I initialized Elite Spark Session
spark = SparkSession.builder.appName('PersonalizedEdu').getOrCreate()

#I generated synthetic Interaction Data (UserID, ContentID, EngagementScore 1-5)
data = [(random.randint(1, 1000),  random.randint(1, 500), random.uniform(1,5)) for _ in range(10000)]
df = spark.createDataFrame(data, ['userId', 'contentId', 'rating'])
df.show(5)


+------+---------+------------------+
|userId|contentId|            rating|
+------+---------+------------------+
|   662|      238| 4.140210511698637|
|   130|       71|1.0316924388696744|
|   737|      210|1.1654720504603233|
|   127|      307|2.3900858555450477|
|   332|      429| 4.642856845689657|
+------+---------+------------------+
only showing top 5 rows


Then I will feature enginner duh.
ALS requires integer IDs. If your data uses UUIDs or Strings (like 'Intro_to_Quantum'), I use StringIndexer

In [7]:
from pyspark.ml.feature import StringIndexer

#Logic: Convert IDs  to indexed integers for the matrix
indexer = StringIndexer(inputCol='contentId', outputCol ='contentIndex')
df_indexed = indexer.fit(df).transform(df)

#Then split my training and test set (80/20)
(train, test) = df_indexed.randomSplit([0.8, 0.2])

Then I use ALS which is industry standard for scalable recommendation

In [9]:
from pyspark.ml.recommendation import ALS

#! configured a cold start "drop" that makes sure i do not get NAN errors when I am evaluating
als = ALS(maxIter=10,
          regParam=0.1,
          userCol='userId',
          itemCol='contentIndex',
          ratingCol='rating',
          coldStartStrategy='drop')
model = als.fit(train)

# Now I evaluated YUH

In [10]:
from pyspark.ml.evaluation import RegressionEvaluator

#Now I Evaluate
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating', predictionCol='prediction')
rmse =evaluator.evaluate(predictions)
print(f'Root-mean-square error = {rmse}')

# NOw I generate my top 3 recommendations for all Users
user_recs = model.recommendForAllUsers(3)
user_recs.show(5, truncate=False)

Root-mean-square error = 1.542115416041767
+------+-----------------------------------------------------+
|userId|recommendations                                      |
+------+-----------------------------------------------------+
|1     |[{448, 4.8689017}, {229, 4.773467}, {147, 4.6904407}]|
|2     |[{25, 4.0337243}, {165, 3.9390948}, {119, 3.8045537}]|
|3     |[{252, 3.8232355}, {140, 3.774166}, {461, 3.6913633}]|
|4     |[{115, 4.1931653}, {128, 3.885631}, {448, 3.867343}] |
|5     |[{14, 4.2737694}, {448, 4.1018558}, {15, 3.9511433}] |
+------+-----------------------------------------------------+
only showing top 5 rows


#Eigentiki's interpretation

#####So like bro imagine a massive spreadsheet. Rows are Users, colums are the puzzles. Most cells are empty because I have not done any puzzles on the Brillant app.

ALS works like a detective. It breaks this big spreadsheet into smaller spreadsheets

User traits and Content traits

By multiplying the hiddent traits back together, the AI fills in the blanks to predict that if I liked a level 1 puzzle in a section I will probably like the level 2 of that puzzle  on the Brilliant app.